In [40]:
import numpy as np
import matplotlib.pyplot as plt
import random
import copy

# Define the problem instance
num_items = 47
bin_capacity = 1000
mutation_rate = 0.001
crossover_rate = 0.6
generations = 10

item_weights = {
    200: 2, 199: 4, 198: 1, 197: 1, 196: 2, 195: 2, 194: 2, 193: 1,
    191: 2, 190: 1, 189: 2, 188: 1, 187: 2, 186: 1, 185: 2, 184: 5,
    183: 1, 182: 1, 181: 3, 180: 2, 179: 2, 178: 1, 176: 1, 175: 2, 
    174: 5, 173: 1, 172: 3, 171: 1, 170: 4, 169: 2,  168: 1, 167: 5,
    165: 2, 164: 2, 163: 3, 162: 2, 160: 2, 159: 2, 158: 2, 157: 4,
    156: 3, 155: 2, 154: 1, 153: 3, 152: 2, 151: 2, 150: 2
}
def binary_to_decimal(binary_list):
    decimal_list = []
    for binary_nums in binary_list:
        decimal_nums = []
        for binary_num in binary_nums:
            if isinstance(binary_num, list):
                binary_num = ''.join([str(bit) for bit in binary_num])
            decimal_num = int(str(binary_num), 2)
            decimal_nums.append(decimal_num)
        decimal_list.append(decimal_nums)
    return decimal_list

def decimal_to_binary(decimal_list):
    bits = ''
    bits_list = []
    for sublist in decimal_list:
        sublist_bits = []
        for num in sublist:
            sublist_bits.append(format(num, '08b'))
        bits_list.append(sublist_bits)

    binary_list = '[' + ', '.join(['[' + ', '.join(sublist) + ']' for sublist in bits_list]) + ']'
    return binary_list
    
def sum_subsets(subsets):
    sums = []
    for subset in subsets:
        subset_sum = sum(subset)
        if subset_sum > bin_capacity:
            subset_sum = 0
        sums.append(subset_sum)
    return sums


def mutate_individual(individual, mutation_rate):
    mutated_individual = individual.copy()
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            # Flip the bit at position i
            mutated_individual[i] = 1 - mutated_individual[i]
    return mutated_individual

# Function for crossover
def crossover(parent1, parent2):
    if np.random.rand() < crossover_rate:
        crossover_point = np.random.randint(1, 7)
        child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
        return child1, child2
    else:
        return parent1, parent2

In [41]:
def pick_random_integer():
    choices = list(item_weights.keys())
    weights = list(item_weights.values())
    random_integer = random.choices(choices, weights=weights)[0]
    return random_integer

def pick_integers():
    chosen_integers = []
    sublists = [[] for _ in range(num_items)]  # Create 47 empty sublists
    for _ in range(len(item_weights)):
        random_integer = pick_random_integer()
        chosen_integers.append(random_integer)
        sublist_index = len(chosen_integers) % num_items  # Determine the index of the sublist
        sublists[sublist_index].append(random_integer)  # Append the integer to the corresponding sublist
        item_weights[random_integer] -= 1  # Decrease the count of the chosen integer
        if item_weights[random_integer] == 0:
            del item_weights[random_integer]  # Remove the integer if its count becomes 0
            
    # Append remaining integers to sublists in sequence
    remaining_integers = list(item_weights.keys())
    for i, integer in enumerate(remaining_integers):
        sublist_index = i % 47
        sublists[sublist_index].append(integer)
        
    return sublists



In [42]:
chosen_sublists = pick_integers()
print("Chosen sublists:", chosen_sublists)

Chosen sublists: [[170, 196], [179, 195], [200, 190], [163, 189], [183, 188], [159, 187], [174, 185], [156, 184], [174, 181], [181, 180], [164, 178], [175, 175], [199, 174], [184, 173], [184, 172], [163, 171], [172, 170], [191, 169], [165, 168], [179, 167], [158, 165], [156, 164], [180, 163], [194, 160], [153, 158], [199, 157], [152, 156], [198, 155], [191, 154], [187, 153], [189, 152], [196, 151], [197, 150], [185], [162], [170], [169], [162], [182], [176], [199], [194], [200], [193], [186], [159], [199]]


In [43]:
# Fuction to calculate fitness
def calculate_fitness(population):
        indiv_fitness = sum_subsets(population)
        return indiv_fitness  

In [44]:
# Main genetic algorithm loop
for generation in range(generations):
    fitness_values = calculate_fitness(chosen_sublists)
    fitness_bin_history = len(chosen_sublists)
    print("Bins used: " + str(fitness_bin_history))
    #fitness_bin_history.append(fitness_values)

   # Convert fitness values to probabilities
    total_fitness = sum(fitness_values)
    probabilities = [fitness / total_fitness for fitness in fitness_values]

    # Select parents based on fitness
    parents_indices = np.random.choice(len(chosen_sublists), size=len(chosen_sublists) // 2, p=probabilities)
    parents = [chosen_sublists[i] for i in parents_indices]

    # Create next generation through crossover and mutation
    children = []
    for i in range(0, 5):
        child1, child2 = crossover(parents[i], parents[i + 1])
        child1 = mutate_individual(child1, mutation_rate)
        child2 = mutate_individual(child2, mutation_rate)
        children.extend([child1, child2])

    chosen_sublists[:len(chosen_sublists) // 2] = parents
    chosen_sublists[len(chosen_sublists) // 2:] = children

Bins used: 47
Bins used: 33
Bins used: 26
Bins used: 23
Bins used: 21
Bins used: 20
Bins used: 20
Bins used: 20
Bins used: 20
Bins used: 20
